In [1]:
## Building A Chatbot

In [2]:
import os 
from dotenv import load_dotenv
load_dotenv() ## loading all the environment variable

True

In [4]:
groq_api_key = os.getenv("GROQ_API_KEY")

'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'

In [5]:
from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001CBBD6023E0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001CBBD5DBE50>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [7]:
from langchain_core.messages import HumanMessage, AIMessage
model.invoke([HumanMessage(content="Hi, My name is Akash and I'm a data scientist")])

AIMessage(content="Hi Akash, it's nice to meet you! \n\nWhat kind of data science work do you do?  Are you working on any interesting projects right now? \n\nI'm always eager to learn more about how people are using data science to solve problems.\n", response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 23, 'total_tokens': 82, 'completion_time': 0.107272727, 'prompt_time': 0.00013453, 'queue_time': 0.01459102, 'total_time': 0.107407257}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-beb639f9-1abe-4452-b049-1b68dd4430f9-0', usage_metadata={'input_tokens': 23, 'output_tokens': 59, 'total_tokens': 82})

In [8]:
## Now the model can remember the previous chat as well

model.invoke(
    [
        HumanMessage(content="Hi, My name is Akash and I'm a data scientist"),
        AIMessage(content="Hi Akash, it's nice to meet you! \n\nWhat kind of data science work do you do?  Are you working on any interesting projects right now? \n\nI'm always eager to learn more about how people are using data science to solve problems.\n", response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 23, 'total_tokens': 82, 'completion_time': 0.107272727, 'prompt_time': 0.00013453, 'queue_time': 0.01459102, 'total_time': 0.107407257}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-beb639f9-1abe-4452-b049-1b68dd4430f9-0', usage_metadata={'input_tokens': 23, 'output_tokens': 59, 'total_tokens': 82})
        , HumanMessage(content="What is my name and what do I do?")
    ]
)

AIMessage(content="You're Akash, and you are a data scientist! \n\nIs there anything else you'd like to tell me about yourself or your work?  I'm ready to listen! 😊  \n", response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 100, 'total_tokens': 145, 'completion_time': 0.081818182, 'prompt_time': 0.007005943, 'queue_time': 0.6180782539999999, 'total_time': 0.088824125}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-fee7f667-37f4-4c68-ae64-bf366b3b6e48-0', usage_metadata={'input_tokens': 100, 'output_tokens': 45, 'total_tokens': 145})

In [ ]:
## Message History

# We can use a Message History Class to wrap our model and make it stateful. This will keep track
# of inputs and output of the model, and store them in some datastore. Future interaction will then load 
# those messages and pass them into the chain as part of the input.

In [9]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


In [13]:

# To separate out one use chat from other user's chat we will use following technique

# Session will create unique chat_id for single user's chathistory

store = {}
def get_session_history(session_id:str)-> BaseChatMessageHistory:
    #Checking if the session id is present or not 
    if session_id not in store:
        # If a session_id is present it will return it entire chat (suppose after login)
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [14]:
# 
config = { "configurable": {"session_id" : "chat1"}}

In [16]:
# Passing the session_id or chat history id
response = with_message_history.invoke(
    [
        HumanMessage(content="Hi, my name is akash and I'm a data scientist")
    ], config=config
)

In [17]:
response.content

"Hi Akash! \n\nIt's nice to meet you.  \n\nWhat kind of data science work do you do?  I'm always interested in learning more about how people use data to solve problems and make discoveries.\n"

In [18]:
with_message_history.invoke(
    [
        HumanMessage(content="What is my name?")
    ], config=config
)

AIMessage(content='Your name is Akash.  😊 \n\nI remember! \n\n\n\n', response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 158, 'total_tokens': 176, 'completion_time': 0.032727273, 'prompt_time': 0.00549474, 'queue_time': 0.008596050000000001, 'total_time': 0.038222013}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-bd4899f7-a405-46bc-bf2d-3f7ebf0744d1-0', usage_metadata={'input_tokens': 158, 'output_tokens': 18, 'total_tokens': 176})

In [21]:
## Change the config --> session id
## This time we are providing the new configue to check whether is remember the old history on not based on the chat id
## If we will put any other session_id then it will not remember the chat history
# 3
config1 = { "configurable": {"session_id" : "chat1"}}
response = with_message_history.invoke(
    [HumanMessage(content="What my name?")], config=config1
)

response.content 

"Your name is Akash. \n\nI understand you might be curious about your name, but I've already told you! 😊 \n\nIs there anything else I can help you with?\n"

In [ ]:
# Prompt Template 

# Prompt templates help to turn raw user information into a format that the LLM can work with. In this 
# case, the raw user input is just a message, which we are passing to the LLM. 

In [26]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","you are helpful assistant. Answer all the question to the best of your abaility"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt|model

In [27]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Akash Patel")]})

AIMessage(content="Hi Akash Patel, it's nice to meet you!  \n\nHow can I help you today? 😊 \n\n", response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 32, 'total_tokens': 60, 'completion_time': 0.050909091, 'prompt_time': 0.000318699, 'queue_time': 0.013297798999999999, 'total_time': 0.05122779}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-a6690659-b9c4-4b6c-aa8d-fbc649acd032-0', usage_metadata={'input_tokens': 32, 'output_tokens': 28, 'total_tokens': 60})

In [28]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [29]:
config = {"configurable":{"session_id": "chat3"}}
with_message_history.invoke(
    [HumanMessage(content="Hi My name is Akash Patel")],config=config
)

response.content

"Your name is Akash. \n\nI understand you might be curious about your name, but I've already told you! 😊 \n\nIs there anything else I can help you with?\n"

In [31]:
## Adding more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
        "you are a helpful assistant. Answer all the question in {language} language.",
        ), MessagesPlaceholder(variable_name="messages"), 
    ]
)

chain = prompt | model

In [32]:
response = chain.invoke({"messages": [HumanMessage(content="Hi My name is Akash")], "language":"Hindi"})
response.content

'नमस्ते अकाश! 🙂  मुझे आपकी मदद करने में खुशी होगी। आपका कोई सवाल है? \n'

In [33]:
# Now wrap this more complicated chain in message history class. This time, because there are
# multiple keys in the input, we need to specify the correct key to use to save the chat history

In [34]:
with_message_history = RunnableWithMessageHistory(
    chain, 
    get_session_history,
    input_messages_key="messages"
)

In [40]:
config5 = { "configurable": {"session_id" : "chat5"}}
response = with_message_history.invoke(
    {'messages' : [HumanMessage(content="Hi, I am Akash Patel")], "language":"Hindi"},
    config=config5
)
response.content

'नमस्ते अकाश पटेल! आपका स्वागत है।  \n\nक्या मैं आपके लिए कुछ कर सकता हूँ? 😊\n'

In [ ]:
## Managing the conversion history

# One import concept to understand when building chatbots is how to manager conversation
# history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM.
# Therefore, it is important to add a step that limits the size of the messages you are passing in.

# trim_messages: this helper to reduce how many messages we're sending to the model. The trimmer allows
# us to specify how many tokens we want to keep, along with other parameter like if we want to always keep the system
# message and whether to allow partial messages

In [43]:
from langchain_core.messages import SystemMessage, trim_messages


In [46]:
trimmer = trim_messages(
    max_tokens = 45,
    # last means focus on the last conversion and from their itself it will count tokens
    strategy = "last",
    token_counter = model,
    include_system = True,
    allow_partial = False,
    start_on = "human"
)


In [47]:
# We can play with the token number to get the number of messages

messages = [
       SystemMessage(content="you're a good assistant"),
       HumanMessage(content="hi, I'm bob "),
       AIMessage(content='hi!'),
       HumanMessage(content='I like vanilla ice cream'),
       AIMessage(content='nice'),
       HumanMessage(content='whats 2 + 2'),
       AIMessage(content='4'),
       HumanMessage(content='thanks'),
       AIMessage(content='no problem!'),
       HumanMessage(content='having fun?'),
       AIMessage(content='yes!')
      ]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [51]:
# how to pass the messages in the chain

from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages = itemgetter('messages')| trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
    "messages":messages + [HumanMessage(content="what icecream do I like")],
    "language": "English"
    }
)
response.content

"As a large language model, I don't have access to your personal information, including your favorite ice cream flavor. \n\nWhat's your favorite flavor? 😊🍦\n"

In [52]:
response = chain.invoke(
    {
    "messages":messages + [HumanMessage(content="what math problem did I asked for")],
    "language": "English"
    }
)
response.content

'You asked what 2 + 2 equals.  😊  \n\n\n\nLet me know if you have any other questions!\n'

In [54]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key = "messages",
)

config = {"configurable": {"session_id" :"chat6" }}

In [58]:
response = chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What is my  math problem did I asked?")],
    "language": "English"
    },
    config=config,
)
response.content

'You asked: "What is 2 + 2?" \n\n\nLet me know if you\'d like to try another one! 😊  \n'